# Baselines

This module describe and compute the baselines used to evaluate our models results.

In [1]:
import pandas as pd
import pymongo

from time import mktime
from dateutil import parser
from time import strptime
from pymongo import MongoClient
from tqdm import tqdm
from collections import Counter
from bson.objectid import ObjectId
from collections import OrderedDict 

from sklearn.preprocessing import MinMaxScaler

import wikipedia
import urllib.request as urlr
import json

client = MongoClient()
db = client.wyscout

# <font color=red> 1. Data manipulation on the collections</font>

## <font color=orange>1.1 create the `baselines` collection with `matchId` and `teamId` fields</font>

In [233]:
def createBaselinesColl (tm_col, bs_col):
    for teamDoc in list(tm_col.find()):
        for k in teamDoc['matchesPlayed'].keys():
            for matchId in teamDoc['matchesPlayed'][k]:
                bs_col.insert_one({ 'matchId': str(matchId), 'teamId': str(teamDoc['wyId']) })

## <font color=orange>1.2 Add `matchesPlayed` subdocument to each `team` document</font>

In [228]:
def addMatchesPlayed (mt_col, pl_col, tm_col, cm_col):
    compIds = [ d['wyId'] for d in cm_col.find() ]
    teamMathces = { str(x):[] for x in compIds }
    tmDocIds = [ e['_id'] for e in tm_col.find() ]
    for i in tqdm(tmDocIds):
        tm_col.update_one({ '_id':i }, { '$set': { 'matchesPlayed': teamMathces } })
    
    for comp in tqdm(compIds):
        for matchDoc in [ e for e in mt_col.find({ 'competitionId': comp }) ]:
            for team in matchDoc['teamsData'].keys():
                tmDoc = tm_col.find_one({ 'wyId': int(team) })
                l = tmDoc['matchesPlayed'][str(comp)]
                l += [ str(matchDoc['wyId']) ]
                tm_col.update_one({ '_id': tmDoc['_id'] }, 
                                  { '$set': { 
                                      'matchesPlayed.{}'.format(str(comp)): l } })
    

In [229]:
addMatchesPlayed (db.matches, db.players, db.teams, db.competitions)


100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:08<00:00,  1.35s/it]

## <font color=orange>1.3 Add `matchesPlayed` subdocument to each `team` document</font>

In [380]:
def addWikiName (tm_col):
    df = pd.read_csv("C:/Users/riky9/Desktop/BDA/data/teamWiki.csv")
    teamIds = [e['wyId'] for e in list(tm_col.find())]
    for team in tqdm(teamIds):
        wikiName = df[ df['wyId']==team ]['wikiname'].values[0]
        tm_col.update_one({'wyId': team}, { '$set': { 'wikiName': wikiName } })

## <font color=orange>1.4 Add `goalsDiff` subdocument to each `match` document</font>

In [242]:
def addGoalsDiffToMatches (mt_col):
    matchIds = [ d['wyId'] for d in mt_col.find() ]
    for match in tqdm(matchIds):
        teamsIds = [ k for k in mt_col.find_one({'wyId': match})['teamsData'].keys() ]
        teamA = str(teamsIds[0])
        teamB = str(teamsIds[1])
        gdA = db.res_1H_2H.find_one({ 'matchId': str(match), 'teamId': teamA })['goalsDiff']
        gdB = db.res_1H_2H.find_one({ 'matchId': str(match), 'teamId': teamB })['goalsDiff']
        mt_col.update_one({ 'wyId':match }, 
                              { '$set': { 'goalsDiff': { teamA: gdA, teamB: gdB } }  })


 72%|████████████████████████████████████████████████████████▌                     | 1407/1941 [00:15<00:07, 73.94it/s]


100%|██████████████████████████████████████████████████████████████████████████████| 1941/1941 [00:23<00:00, 55.89it/s]

## <font color=orange>1.5 Add `wikiName` fiels to each `match` document</font>

In [330]:
def addMorePowerfulTeam (mt_col):
    matchIds = [ d['wyId'] for d in mt_col.find() ]
    for match in tqdm(matchIds):
        matchDoc = mt_col.find_one({ 'wyId': match })
        teamIds = list(matchDoc['teamPlayerank'].keys())
        teamA = teamIds[0]
        teamB = teamIds[1]
        best = teamA
        if matchDoc['teamPlayerank'][teamB] > matchDoc['teamPlayerank'][teamA]:
            best = teamB
        elif matchDoc['teamPlayerank'][teamB] == matchDoc['teamPlayerank'][teamA]:
            best = 0
        mt_col.update_one({ 'wyId': match }, { '$set': { 'morePowerfulTeam': best } })

## <font color=orange>1.6 Add `teamPlayerank` and `timestamp` fields to each `baselines` document</font>

In [ ]:
def addTmpAndTeamPlayerank (mt_col, bs_col):
    matchIds = [ d['wyId'] for d in mt_col.find() ]
    for match in tqdm(matchIds):
        matchDoc = mt_col.find_one({'wyId': match})
        teamsIds = [ k for k in matchDoc['teamsData'].keys() ]
        for team in teamsIds:
            bs_col.update_many({ 'matchId': str(match) }, 
                                     {'$set': {
                                         'timestamp': matchDoc['timestamp'],
                                         'teamPlayerank': matchDoc['teamPlayerank'][str(team)] 
                                     } })

# <font color=red> 2. Compute baselines</font>

A document of the `baselines` collection will be composed by the following fields:

- `matchId`: corresponds to the field `wyId` of each document in the `matches` collection
- `teamId`: corresponds to the field `wyId` of each document in the `teams` collection
- `timestamp`: corresponds to the timespat of the field `data` stored in the document of that match
- `teamPlayerank`: corresponds to the team player rank evaluated for that team with all the matches played before the actual match

Then at each document are added a field for each one of the baseline implemented to predict the different target variables:


1. To predict the __first target variable__, `isWinner`, we have implemented a baseline:

    __1.1__ `baseline1_1` - to say if a team wins a match based on its goodness (`morePowerful`).
    
    __1.2.__ `baseline1_2` - to say if a team _A_ wins a match against a team _B_ more/less powerful, analyzing all the matches in which _A_ won against a team more/less powerful.
    
    
2. To predict the __second target variable__, `didScoreInTWP`, we have implemented a baseline:
    
    __2.1.__ `baseline2_1` - to say if a team did a score in the time window of prediction analyzing the same time window of all the matches available.
    
    
3. To predict the __third target variable__, `goalsDiff`, we have implemented a baseline:
    
    __3.1.__ `baseline3_1` - to compute the difference between the number of goals scored and conceded for a team _A_ that play a match against a team _B_ more/less powerful, analyzing all the matches in which A have played against a team more/less powerful before the date of the actual match.
    
    __3.2.__ `baseline3_2` - to compute the difference between the number of goals scored and conceded for a team _A_ that play a match against a team _B_, analyzing all the matches played by _A_ before the date of the actual match.

## <font color=orange>Add baseline `1.1` and `3.2` </font>


The baseline is stored in the `baselines` collection with the following field:
- `baseline1_1`: is the mode value of all the values stored in the field `winner` of all the matches played by that team before this match. Moreover if the mode is the same for winning and losing matches, we select the mode given a more weight to the matches that the team has won in the same `side` of the actual match. If, again, the mode is the same we set this field as 0. __Note:__ it's not possible to evaluate the mode value of the previous matches played by a team if the actual match is the fist of the competition for that team, so in that case the filed has _None_ value.


- `baseline3_2`: is the average value of all the values stored in the field `goalsDiff` of all the matches played by that team before this match. __Note:__ it's not possible to evaluate the average value of the previous matches played by a team if the actual match is the fist of the competition for that team, so in that case the filed has _None_ value.

In [332]:
def addBaseline1_1And3_2 (mt_col, tm_col, bs_col):
    matchIds = [ d['wyId'] for d in mt_col.find() ]
    for match in tqdm(matchIds):
        matchDoc = mt_col.find_one({ 'wyId': match })
        tmpAct = matchDoc['timestamp']
        comAct = matchDoc['competitionId']
        best = matchDoc['morePowerfulTeam']
        for team in matchDoc['teamsData'].keys():
            teamDoc = tm_col.find_one({ 'wyId': int(team) })
            teamSide = matchDoc['teamsData'][team]['side']
            teamMatches = [ int(m) for m in teamDoc['matchesPlayed'][str(comAct)] ]
            prevMatchesDocs = [ d for d in list( mt_col.find( { 
                'wyId': { '$in': teamMatches }, 
                'timestamp': { '$lt': tmpAct }} )) ]
            bs1_1 = None
            bs3_2 = None
            # we coudn't evaluate the mean of the goalsDiff of previous matches played a team if 
            # the actual match is the first of the competition for that team
            if len(prevMatchesDocs) > 0:
                try:
                    bs1_1 = mode([ 1 if ((str(d['winner']) == team) & (str(d['morePowerful']) == team)) 
                                  else 0 for d in prevMatchesDocs ])
                except:
                    try:
                        bs1_1 = mode([ 1 if ((str(d['winner']) == team) & (str(d['morePowerful']) == team) 
                                             & (d['teamsData'][team]['side'] == teamSide)) 
                                      else 0 for d in prevMatchesDocs ])
                    except:
                        bs1_1 = 0
                prevGoalsDiff = [ d['goalsDiff'][team] for d in prevMatchesDocs ]
                bs3_2 = round(sum(prevGoalsDiff)/len(prevGoalsDiff))
            
            bs_col.update_one({ 'matchId': str(match), 'teamId': str(team) }, 
                              { '$set': { 'baseline1_1': bs1_1, 'baseline3_2': bs3_2 } })

In [333]:
addBaseline1_1And3_2(db.matches, db.teams, db.baselines)


 27%|█████████████████████▏                                                         | 521/1941 [00:16<00:44, 31.75it/s]


 55%|██████████████████████████████████████████▊                                   | 1065/1941 [00:32<00:24, 36.27it/s]


 82%|████████████████████████████████████████████████████████████████▎             | 1601/1941 [00:49<00:12, 27.12it/s]


100%|██████████████████████████████████████████████████████████████████████████████| 1941/1941 [00:59<00:00, 32.71it/s]

## <font color=orange>Add baseline `1.2` and `3.1` </font>


The two baselines are stored in the `baselines` collection respectively with the following two fields:

- `baseline1_2`: is the mode value of all the values stored in the field `winner` of all the matches played by that team before this match taking into account that: if that match is more (resp. less) powerful than the opponent we select all the previous matches in which that team has played as the more (resp. less) powerful. Moreover if the mode is the same for winning and losing matches, we select the mode given a more weight to the matches that the team has won in the same `side` of the actual match. If, again, the mode is the same we set this field as 0. __Note:__ it's not possible to evaluate the mode value of the previous matches played by a team if the actual match is the fist of the competition for that team, so in that case the filed has _None_ value.


- `baseline3_1`: is the average value of all the values stored in the field `goalsDiff` of all the matches played by that team before this match taking into account that: if that match is more (resp. less) powerful than the opponent we select all the previous matches in which that team has played as the more (resp. less) powerful. __Note:__ it's not possible to evaluate the average value of the previous matches played by a team if the actual match is the fist of the competition for that team, so in that case the filed has _None_ value.

In [327]:
def addBaseline1_2And3_1 (mt_col, tm_col, bs_col):
    matchIds = [ d['wyId'] for d in mt_col.find() ]
    for match in tqdm(matchIds):
        matchDoc = mt_col.find_one({ 'wyId': match })
        tmpAct = matchDoc['timestamp']
        comAct = matchDoc['competitionId']
        best = matchDoc['morePowerfulTeam']
        for team in matchDoc['teamsData'].keys():
            teamDoc = tm_col.find_one({ 'wyId': int(team) })
            teamSide = matchDoc['teamsData'][team]['side']
            teamMatches = [ int(m) for m in teamDoc['matchesPlayed'][str(comAct)] ]
            if team == best: #case in which that team is the more powerful
                prevMatchesDocs = [ d for d in list( mt_col.find( { 
                    'wyId': { '$in': teamMatches }, 
                    'timestamp': { '$lt': tmpAct }, 
                    'morePowerfulTeam': str(team) } )) ]
            else: #case in which that team is the less powerful
                prevMatchesDocs = [ d for d in list( mt_col.find( { 
                    'wyId': { '$in': teamMatches }, 
                    'timestamp': { '$lt': tmpAct }, 
                    'morePowerfulTeam': { '$ne': str(team) } } )) ]
            bs1_2 = None
            bs3_1 = None
            # we coudn't evaluate the mean of the goalsDiff of previous matches played a team if 
            # the actual match is the first of the competition for that team
            if len(prevMatchesDocs) > 0:
                try:
                    bs1_2 = mode([ 1 if str(d['winner']) == team else 0 for d in prevMatchesDocs ])
                except:
                    try:
                        bs1_2 = mode([ 1 if ((str(d['winner']) == team) & (d['teamsData'][team]['side'] == teamSide)) 
                                      else 0 for d in prevMatchesDocs ])
                    except:
                        bs1_2 = 0
                prevGoalsDiff = [ d['goalsDiff'][team] for d in prevMatchesDocs ]
                bs3_1 = round(sum(prevGoalsDiff)/len(prevGoalsDiff))
            
            bs_col.update_one({ 'matchId': str(match), 'teamId': str(team) }, 
                              { '$set': { 'baseline1_2': bs1_2, 'baseline3_1': bs3_1 } })

In [328]:
addBaseline1_2And3_1(db.matches, db.teams, db.baselines)


 33%|██████████████████████████▏                                                    | 642/1941 [00:15<00:34, 37.71it/s]


 68%|█████████████████████████████████████████████████████▏                        | 1324/1941 [00:31<00:17, 35.39it/s]


100%|██████████████████████████████████████████████████████████████████████████████| 1941/1941 [00:45<00:00, 42.78it/s]

## <font color=orange>Add baseline `2.1` </font>


The baseline is stored in the `baselines` collection with the following field:
- `baseline2_1`: is a dictionary which the key corresponds to a string `x_y` where _x_ are respectively the minutes of the analized and the predicted time window. For each key the value is  the the mode value of all the values stored in the field `didScoreInTWP` of all the matches played by that team before this match and for the same time widnow analyzed/predicted. Moreover if the mode is the same for winning and losing matches, we select the mode given a more weight to the matches that the team has won in the same `side` of the actual match. If, again, the mode is the same we set this field as 0. __Note:__ it's not possible to evaluate the mode value of the previous matches played by a team if the actual match is the fist of the competition for that team, so in that case the filed has _None_ value.

In [2]:
def addBaseline2_1 (mt_col, tm_col, bs_col):
    minTWs = [(5, 5), (10, 5), (10, 10), (20, 5), (20, 10), (20, 20), (30, 5), (30, 10), (30, 20), (30, 30), 
              (45, 5), (45, 10), (45, 20), (45, 30), (45, 45), (60, 5), (60, 10), (60, 20), (60, 30), 
              (75, 5), (75, 10), (75, 15)]
    
    matchIds = [ d['wyId'] for d in mt_col.find() ]
    
    for x, y in tqdm(minTWs):
        minStr = str(x) + '_' + str(y)
        nameResCol = 'db.res_' + minStr
        res_col = eval(nameResCol)
    
        for match in tqdm(matchIds):
            matchDoc = mt_col.find_one({ 'wyId': match })
            tmpAct = matchDoc['timestamp']
            comAct = matchDoc['competitionId']
            
            for team in matchDoc['teamsData'].keys():
                teamDoc = tm_col.find_one({ 'wyId': int(team) })
                teamSideHome = int(matchDoc['teamsData'][team]['side'] == 'home')
                teamMatches = [ int(m) for m in teamDoc['matchesPlayed'][str(comAct)] ]

                prevMatchesIds = [ str(d['wyId']) for d in list( mt_col.find( { 
                    'wyId': { '$in': teamMatches }, 
                    'timestamp': { '$lt': tmpAct }} )) ]
                
                prevResDocs = [ d for d in list( res_col.find( { 'matchId': { '$in': prevMatchesIds } } )) ]
                
                bs2_1 = None
                # we coudn't evaluate the mean of the goalsDiff of previous matches played a team if 
                # the actual match is the first of the competition for that team
                if len(prevResDocs) > 0:
                    try:
                        bs2_1 = mode([ d['didScoreInTWP'] for d in prevResDocs ])
                    except:
                        try:
                            bs2_1 = mode([ d['didScoreInTWP'] for d in prevResDocs 
                                          if int(d['teamsData'][team]['side'] == 'home') == teamSideHome ])
                        except:
                            bs2_1 = 0

                bs_col.update_one({ 'matchId': str(match), 'teamId': str(team) }, 
                                  { '$set': { 'baseline2_1.{}'.format(minStr): bs2_1 } })

In [3]:
addBaseline2_1(db.matches, db.teams, db.baselines)

 22%|█████████████████                                                              | 419/1941 [00:16<00:59, 25.42it/s]


 48%|█████████████████████████████████████▊                                         | 929/1941 [00:33<00:37, 27.23it/s]


 70%|██████████████████████████████████████████████████████▌                       | 1358/1941 [00:50<00:23, 25.23it/s]


 94%|█████████████████████████████████████████████████████████████████████████     | 1819/1941 [01:07<00:03, 34.26it/s]


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [01:10<00:00, 70.30s/it]
